# Simulation of CNCS powder experiment

In [ ]:
# some goodies
# %matplotlib notebook
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np, histogram.hdf as hh, histogram as H

## Create workflow

In [ ]:
workdir = "~/simulations/CNCS/powder-demo"
!mkdir -p {workdir}
%cd {workdir}

In [ ]:
# rm -rf mysim/

In [ ]:
!mcvine workflow powder --instrument=CNCS --sample=V --workdir=mysim

In [ ]:
!which mcvine

In [ ]:
!ls mysim/

## Beam simulation

In [ ]:
# change to beam sim directory
%cd {workdir}/mysim/beam

In [ ]:
# show the simulation script
!cat run-beam.sh

In [ ]:
# modify the simulation script (desired incident energy, neutron count)

In [ ]:
%%file run-beam.sh
#!/usr/bin/env bash

mcvine instruments cncs beam --keep-in-cache --use-cache -E=12 --ncount=1e9 --nodes=25

In [ ]:
# show the simulation script again
!cat run-beam.sh

In [ ]:
%%time
# run simulation
!./run-beam.sh

After the beam simulation is done
* The main output are in the new folder "out".
* The ".h5" files are histograms of various sorts.
* The "neutron" file is the simulated neutrons of the beam incident on the sample

In [ ]:
!ls out

In [ ]:
# plot I(E) spectrum
ie = hh.load("out/ienergy.h5")
plt.plot(ie.energy, ie.I)

## Scattering simulation and reduction

The simulation directory contains a Makefile and various scripts to make it easy to run the simulation and analsysis.

In [ ]:
%cd {workdir}/mysim
!ls

In [ ]:
# scatter is the script for sample scattering simulation
!cat scatter

In [ ]:
# Change ncount and nodes

In [ ]:
%%file scatter
#!/usr/bin/env bash

LAUNCHER=`mcvine mpi launcher`
rm -rf out && ./sss --ncount=1e7 --multiple-scattering=off --$LAUNCHER.nodes=10 --buffer_size=100000


In [ ]:
# print beam simulation information
!cat {workdir}/mysim/beam/out/props.json

** !!! Modify the following cell by setting ei_guess and t0_guess to values printed out in the cell above !!! **

In [ ]:
%%file reduce2iqe
#!/usr/bin/env bash

time mcvine instruments arcs nxs reduce sim.nxs --qaxis=${QAXIS} --tof2E  --ei_guess=12.1190218495 --t0_guess=135.846586772

In [ ]:
! make clean

In [ ]:
%%time
# run simulation and reduction
!chmod +x scatter sss reduce2iqe create-nxs
!make

## Plot results

In [ ]:
iqe = hh.load("iqe.h5")

In [ ]:
H.plot(iqe[(0,5), ()], min=0, max=1e-5)

In [ ]:
iqe2 = iqe.copy()
iqe2.I[iqe2.I!=iqe2.I] = 0
ie = iqe2.sum('Q')

In [ ]:
plt.plot(ie.E, ie.I)
plt.xlim(-3, 2)